In [1]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [2]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [3]:
# imports
import pprint
import pickle
import glob
import random
from pathlib import Path
import math

import torch
from tqdm import tqdm
from PIL import Image
import numpy as np
from pytorch3d.renderer import look_at_view_transform
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from utils import general_utils
from utils import eval_utils
import deformation.losses as def_losses

In [4]:
def compare_before_after_refinement(results_df_before, results_df_after):
    
    statistics = {"num_3d_iou_improved":0, "avg_improve_amt": 0, "avg_improve_before_2d_iou":0, "num_3d_iou_worse":0,  "avg_worse_amt":0, "avg_worse_before_2d_iou":0}
    improved_instances = []
    worse_instances = []
    comparision_df = pd.DataFrame()
    for instance in list(results_df_after["instance"]):
        if instance in list(results_df_before["instance"]):
            before_result = results_df_before[results_df_before["instance"]==instance]
            before_3d_iou = float(before_result["3d_iou"])
            #before_2d_iou = float(before_result["2d_iou"])
            after_result = results_df_after[results_df_after["instance"]==instance]
            after_3d_iou = float(after_result["3d_iou"])
            #after_2d_iou = float(after_result["2d_iou"])

            #iou_2d_change = after_2d_iou - before_2d_iou
            iou_3d_change = after_3d_iou - before_3d_iou

            if math.isnan(iou_3d_change):
                statistics["num_3d_iou_worse"] += 1
                continue
            elif iou_3d_change > 0:
                statistics["num_3d_iou_improved"] += 1
                statistics["avg_improve_amt"] += iou_3d_change
                #statistics["avg_improve_before_2d_iou"] += before_2d_iou
                improved_instances.append(instance)
            else:
                statistics["num_3d_iou_worse"] += 1
                statistics["avg_worse_amt"] += iou_3d_change
                #statistics["avg_worse_before_2d_iou"] += before_2d_iou
                worse_instances.append(instance)
            comparision_df = comparision_df.append({"instance": instance, "iou_3d_delta": iou_3d_change}, ignore_index=True)

    statistics["avg_improve_amt"] /= statistics["num_3d_iou_improved"]
    statistics["avg_improve_before_2d_iou"] /= statistics["num_3d_iou_improved"]
    statistics["avg_worse_amt"] /= statistics["num_3d_iou_worse"]
    statistics["avg_worse_before_2d_iou"] /= statistics["num_3d_iou_worse"]
            
    pprint.pprint(statistics)
    #print("example improved instances")
    #print(improved_instances[:100])
    #print("example worse instances")
    #print(worse_instances[:100])
    return comparision_df

In [5]:
def filter_eval_df_warnings(original_df, warnings_to_filter=["NaN nodes", "compute_iou_3d: gt mesh < 200 occupancies", "compute_iou_3d: rec mesh < 200 occupancies"]):
    filter_array = []
    warnings_list = original_df["eval_warnings"].tolist()
    for i in range(len(original_df)):
        warnings = warnings_list[i]
        keep=True
        for warning in warnings:
            if warning in warnings_to_filter:
                keep=False
        filter_array.append(keep)
    return original_df[filter_array]

def filter_and_sort_eval_dfs(df1, df2):
    df1 = filter_eval_df_warnings(df1)
    df2 = filter_eval_df_warnings(df2)
    df1_instances = df1["instance"].tolist()
    df2_instances = df2["instance"].tolist()
    
    df1_filters = [(True if df1_instance in df2_instances else False) for df1_instance in df1_instances]
    df1_filtered = df1[df1_filters].sort_values(by="instance")
    
    df2_filters = [(True if df2_instance in df1_instances else False) for df2_instance in df2_instances]
    df2_filtered = df2[df2_filters].sort_values(by="instance")
    
    return df1_filtered, df2_filtered

def update_2d_iou_input_to_bf(original_df, bf_2d_iou_df):
    if (original_df["instance"].tolist() != bf_2d_iou_df["instance"].tolist() ):
        raise ValueError("Instances don't match")
    original_df["2d_iou_input"] = bf_2d_iou_df["2d_iou_input"]
    return original_df
    

In [37]:

metrics = ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]
column_order = ["instance"]
for metric in metrics:
    column_order.append("{}_Before".format(metric))
    column_order.append("{}_After".format(metric))
    column_order.append("{}_Delta".format(metric))
pd.options.mode.chained_assignment = None 

def get_processed_combined_df(eval_results_path_before, eval_results_path_after, remove_low_iou=True):
    results_df_before_raw = pd.read_pickle(eval_results_path_before)
    results_df_after_raw = pd.read_pickle(eval_results_path_after)

    results_df_before, results_df_after = filter_and_sort_eval_dfs(results_df_before_raw, results_df_after_raw)
    results_df_before = results_df_before.rename(columns={"2d_iou_input":"2D_IoU", "2d_iou_multi":"MV_IoU", "3d_iou":"3D_IoU", "chamfer_L1":"CD"})
    results_df_after = results_df_after.rename(columns={"2d_iou_input":"2D_IoU", "2d_iou_multi":"MV_IoU", "3d_iou":"3D_IoU", "chamfer_L1":"CD"})
    results_df_after = results_df_after.reset_index(drop=True)
    results_df_before = results_df_before.reset_index(drop=True)
    results_df_before["CD"] = results_df_before["CD"] * 10
    results_df_after["CD"] = results_df_after["CD"] * 10
    if remove_low_iou:
        results_df_before["3D_IoU"] = results_df_before["3D_IoU"].apply(lambda x: (np.nan if x < 0.01 else x))
    results_df_after["3D_IoU"][np.isnan(results_df_before["3D_IoU"])] = np.nan
    
    metrics = ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]
    delta_df = results_df_after[metrics] / results_df_before[metrics]
    delta_df = delta_df.replace([np.inf, -np.inf], np.nan)
    
    #print("Note: filtering results from {} to {}\n".format(max(len(results_df_before_raw), len(results_df_after_raw)), len(results_df_after)))
    
    results_df_before = results_df_before.rename(columns={metric:"{}_Before".format(metric) for metric in metrics})
    results_df_before = results_df_before[["instance"]+["{}_Before".format(metric) for metric in metrics]]
    results_df_after = results_df_after.rename(columns={metric:"{}_After".format(metric)  for metric in metrics})
    results_df_after = results_df_after[["{}_After".format(metric) for metric in metrics]]
    delta_df = delta_df.rename(columns={metric:"{}_Delta".format(metric) for metric in metrics})
    delta_df = delta_df[["{}_Delta".format(metric) for metric in metrics]]
    
    combined_df = pd.concat([results_df_before, results_df_after, delta_df], axis=1)[column_order]
    
    return combined_df

def make_df_readable(original_df):
    readable_df = original_df.copy()
    readable_df.loc['mean'] = readable_df.mean()
    readable_df = readable_df.round(2)
    for column_name in original_df.columns.to_list():
        if ("_D" in column_name) or ("_C" in column_name):
            readable_df[column_name] = readable_df[column_name].apply(lambda x: "+{}%".format(round((x-1)*100)) if (x-1) > 0 else  "{}%".format(round((x-1)*100)))
        elif ("_P_" in column_name):
            readable_df[column_name] = readable_df[column_name].apply(lambda x: "{}%".format(round((x)*100)))
    return readable_df

def print_latex(input_df):
    for index, row in input_df.iterrows():
        latex_exp = " & ".join([str(x) for x in row.to_list()])
        latex_exp += " \\\\"
        latex_exp = latex_exp.replace("%", "\\%").replace("nan", "Mean")
        print(latex_exp)

In [7]:
# shapenet
shapenet_class_ids = ["02691156", "02828884", "02933112", "02958343", "03001627", "03211117", "03636649", "03691459", "04090263", "04256520", "04379243", "04401088", "04530566"]
shapenet_id_to_name = {"02691156":"Airplane", "02828884":"Bench", "02933112":"Cabinet", "02958343":"Car", "03001627":"Chair", "03211117":"Display", "03636649":"Lamp",
                       "03691459":"Speakers", "04090263":"Rifle", "04256520":"Sofa", "04379243":"Table", "04401088":"Telephone", "04530566":"Watercraft"}
pix3d_class_ids = ["bed", "bookcase", "chair", "desk", "misc", "sofa", "table","tool", "wardrobe"]
pix3d_id_to_name = {pix3d_id:pix3d_id.capitalize() for pix3d_id in pix3d_class_ids}

method_paths_dict = {
    "atlasnet":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/AtlasNet/data/pytorch_3d_render_recs/{}",
        "eval_dir_temp_after": "data/refinements/shapenet_atlasnet_refinements/{}/{}",
        "class_ids": shapenet_class_ids,
        "name_cvt_dict": shapenet_id_to_name,
        "available_settings":["gt_pose"]
    },
    "pix2mesh":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/Pixel2Mesh/rec_files/pytorch3d_shapenet_renders/{}/rgba",
        "eval_dir_temp_after": "data/refinements/shapenet_pix2mesh_refinements/{}/{}",
        "class_ids": shapenet_class_ids,
        "name_cvt_dict": shapenet_id_to_name,
        "available_settings":["gt_pose"]
    },
    "occnet":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/occupancy_networks/out/pytorch3d_renders/{}/generation/meshes",
        "eval_dir_temp_after": "data/refinements/shapenet_occnet_refinements/{}/{}",
        "class_ids": shapenet_class_ids,
        "name_cvt_dict": shapenet_id_to_name,
        "available_settings":["gt_pose", "bf_pose"]
    },
    "occnet_pix3d":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/occupancy_networks/out/pix3d/{}/generation/meshes",
        "eval_dir_temp_after": "data/refinements/pix3d_occnet_refinements/{}/{}",
        "class_ids": pix3d_class_ids,
        "name_cvt_dict": pix3d_id_to_name,
        "available_settings":["gt_pose"]
    }
}

In [76]:
print(" & ".join([shapenet_id_to_name[i] for i in shapenet_id_to_name]))

Airplane & Bench & Cabinet & Car & Chair & Display & Lamp & Speakers & Rifle & Sofa & Table & Telephone & Watercraft


In [38]:
# table 1: all three methods side by side on shapenet with gt pose, only percentage change info
dfs_to_combine = []
for method in ["atlasnet", "pix2mesh", "occnet"]:
    quant_results_df_mean = pd.DataFrame()
    setting = "gt_pose"
    for class_id in shapenet_class_ids:
        eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
        eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

        combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after)

        quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
        quant_result_dict_mean["instance"] = shapenet_id_to_name[class_id]
        quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

    readable_quant_results_df_mean = make_df_readable(quant_results_df_mean[column_order])
    if len(dfs_to_combine) == 0:
        readable_quant_results_df_mean = readable_quant_results_df_mean[["instance"]+["{}_Delta".format(metric) for metric in metrics]]
    else:
        readable_quant_results_df_mean = readable_quant_results_df_mean[["{}_Delta".format(metric) for metric in metrics]]
    readable_quant_results_df_mean = readable_quant_results_df_mean.rename(columns={"{}_Delta".format(metric):"{}_{}".format(method, metric) for metric in metrics})
    dfs_to_combine.append(readable_quant_results_df_mean)
    
df_order = ["instance"]+["{}_{}".format(method, metric) for method in ["occnet", "pix2mesh", "atlasnet"]  for metric in metrics]
methods_combined_df = pd.concat(dfs_to_combine, axis=1)[df_order]
display(methods_combined_df)
print_latex(methods_combined_df)

,instance,occnet_2D_IoU,occnet_MV_IoU,occnet_3D_IoU,occnet_CD,pix2mesh_2D_IoU,pix2mesh_MV_IoU,pix2mesh_3D_IoU,pix2mesh_CD,atlasnet_2D_IoU,atlasnet_MV_IoU,atlasnet_3D_IoU,atlasnet_CD
0,Airplane,+55%,+17%,+19%,-35%,+60%,+25%,+183%,-54%,+44%,+21%,+121%,-59%
1,Bench,+20%,+7%,+12%,-5%,+18%,+6%,+60%,-25%,+27%,+7%,+44%,-23%
2,Cabinet,+8%,+2%,+2%,-8%,+10%,+5%,+4%,-20%,+15%,+6%,+18%,-33%
3,Car,+18%,+7%,+11%,-10%,+9%,+7%,+19%,-28%,+48%,+25%,+77%,-75%
4,Chair,+13%,+5%,+20%,-16%,+11%,+8%,+65%,-31%,+14%,+7%,+55%,-46%
5,Display,+14%,+6%,+60%,-27%,+12%,+9%,+49%,-34%,+15%,+5%,+43%,-26%
6,Lamp,+43%,+16%,+16%,-14%,+17%,+8%,+98%,-36%,+39%,+16%,+97%,-47%
7,Speakers,+10%,+3%,+11%,-9%,+8%,+5%,+11%,-15%,+15%,+6%,+21%,-35%
8,Rifle,+62%,+13%,+18%,-24%,+26%,+30%,+125%,-52%,+56%,+30%,+121%,-64%
9,Sofa,+12%,+3%,+7%,+55%,+10%,+2%,+28%,-15%,+19%,+5%,+20%,-36%


Airplane & +55\% & +17\% & +19\% & -35\% & +60\% & +25\% & +183\% & -54\% & +44\% & +21\% & +121\% & -59\% \\
Bench & +20\% & +7\% & +12\% & -5\% & +18\% & +6\% & +60\% & -25\% & +27\% & +7\% & +44\% & -23\% \\
Cabinet & +8\% & +2\% & +2\% & -8\% & +10\% & +5\% & +4\% & -20\% & +15\% & +6\% & +18\% & -33\% \\
Car & +18\% & +7\% & +11\% & -10\% & +9\% & +7\% & +19\% & -28\% & +48\% & +25\% & +77\% & -75\% \\
Chair & +13\% & +5\% & +20\% & -16\% & +11\% & +8\% & +65\% & -31\% & +14\% & +7\% & +55\% & -46\% \\
Display & +14\% & +6\% & +60\% & -27\% & +12\% & +9\% & +49\% & -34\% & +15\% & +5\% & +43\% & -26\% \\
Lamp & +43\% & +16\% & +16\% & -14\% & +17\% & +8\% & +98\% & -36\% & +39\% & +16\% & +97\% & -47\% \\
Speakers & +10\% & +3\% & +11\% & -9\% & +8\% & +5\% & +11\% & -15\% & +15\% & +6\% & +21\% & -35\% \\
Rifle & +62\% & +13\% & +18\% & -24\% & +26\% & +30\% & +125\% & -52\% & +56\% & +30\% & +121\% & -64\% \\
Sofa & +12\% & +3\% & +7\% & +55\% & +10\% & +2\% & +28\% & -15\% & +1

In [39]:
# table 2: occnet on pix3d with gt pose, only percentage change info

method = "occnet_pix3d"
quant_results_df_mean = pd.DataFrame()
setting = "gt_pose"
for class_id in pix3d_class_ids:
    eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
    eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

    combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after)

    quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
    quant_result_dict_mean["instance"] = class_id.capitalize()
    quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

readable_quant_results_df_mean = make_df_readable(quant_results_df_mean[column_order])
readable_quant_results_df_mean = readable_quant_results_df_mean[["instance"]+["{}_Delta".format(metric) for metric in metrics]]
readable_quant_results_df_mean = readable_quant_results_df_mean.rename(columns={"{}_Delta".format(metric):"{}_{}".format(method, metric) for metric in metrics})
    
display(readable_quant_results_df_mean)
print_latex(readable_quant_results_df_mean)

,instance,occnet_pix3d_2D_IoU,occnet_pix3d_MV_IoU,occnet_pix3d_3D_IoU,occnet_pix3d_CD
0,Bed,+55%,+19%,+24%,-19%
1,Bookcase,+70%,+28%,+8%,-29%
2,Chair,+74%,+16%,+22%,-13%
3,Desk,+44%,+10%,+9%,+46%
4,Misc,+102%,+39%,+26%,-33%
5,Sofa,+37%,+8%,+8%,+9%
6,Table,+40%,+12%,+22%,+2%
7,Tool,+89%,+18%,+34%,-36%
8,Wardrobe,+53%,+15%,+7%,-2%
mean,NaN,+63%,+19%,+18%,-8%


Bed & +55\% & +19\% & +24\% & -19\% \\
Bookcase & +70\% & +28\% & +8\% & -29\% \\
Chair & +74\% & +16\% & +22\% & -13\% \\
Desk & +44\% & +10\% & +9\% & +46\% \\
Misc & +102\% & +39\% & +26\% & -33\% \\
Sofa & +37\% & +8\% & +8\% & +9\% \\
Table & +40\% & +12\% & +22\% & +2\% \\
Tool & +89\% & +18\% & +34\% & -36\% \\
Wardrobe & +53\% & +15\% & +7\% & -2\% \\
Mean & +63\% & +19\% & +18\% & -8\% \\


In [41]:
# table 3: occnet on shapenet bf pose, also showing percentage/amt up

method = "occnet"
setting = "bf_pose"
quant_results_df_mean = pd.DataFrame()
for class_id in shapenet_class_ids:
    eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
    eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

    combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after, False)
    quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
    quant_result_dict_mean["instance"] = shapenet_id_to_name[class_id]

    for metric in ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]:
        if metric in ["CD"]:
            num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
            avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean()
            num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
            avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
        else:
            num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
            avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
            num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
            avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean()

        quant_result_dict_mean["{}_P_up".format(metric)] =  num_improved/(num_improved+num_worse)
        quant_result_dict_mean["{}_C_up".format(metric)] =  avg_improve_amt
        quant_result_dict_mean["{}_P_d".format(metric)] =  num_worse/(num_improved+num_worse)
        quant_result_dict_mean["{}_C_d".format(metric)] =  avg_worse_amt


    quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Before".format(metric): "{}_B".format(metric) for metric in metrics})
quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_After".format(metric): "{}_A".format(metric) for metric in metrics})
quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Delta".format(metric): "{}_C".format(metric) for metric in metrics})
readable_quant_results_df_mean = make_df_readable(quant_results_df_mean)
df_order = ["instance"]+["{}_{}".format(metric, i) for metric in ["MV_IoU", "3D_IoU", "CD"] for i in ["P_up", "C_up", "C"]]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(readable_quant_results_df_mean[df_order])

print_latex(readable_quant_results_df_mean[df_order])


,instance,MV_IoU_P_up,MV_IoU_C_up,MV_IoU_C,3D_IoU_P_up,3D_IoU_C_up,3D_IoU_C,CD_P_up,CD_C_up,CD_C
0,Airplane,45%,+12%,-1%,23%,+128%,+5%,37%,-29%,+58%
1,Bench,39%,+8%,-1%,32%,+83%,0%,25%,-20%,+74%
2,Cabinet,34%,+2%,-3%,25%,+25%,-4%,26%,-13%,+68%
3,Car,41%,+4%,-1%,39%,+14%,-2%,36%,-14%,+28%
4,Chair,59%,+5%,+2%,44%,+146%,+50%,55%,-22%,+5%
5,Display,30%,+4%,-6%,37%,+73%,+7%,29%,-25%,+91%
6,Lamp,62%,+13%,+5%,40%,+94%,+22%,49%,-23%,+15%
7,Speakers,47%,+3%,-1%,34%,+24%,-1%,32%,-15%,+44%
8,Rifle,19%,+19%,-14%,21%,+91%,-13%,32%,-35%,+158%
9,Sofa,35%,+5%,-2%,26%,+14%,-10%,26%,-17%,+115%


Airplane & 45\% & +12\% & -1\% & 23\% & +128\% & +5\% & 37\% & -29\% & +58\% \\
Bench & 39\% & +8\% & -1\% & 32\% & +83\% & 0\% & 25\% & -20\% & +74\% \\
Cabinet & 34\% & +2\% & -3\% & 25\% & +25\% & -4\% & 26\% & -13\% & +68\% \\
Car & 41\% & +4\% & -1\% & 39\% & +14\% & -2\% & 36\% & -14\% & +28\% \\
Chair & 59\% & +5\% & +2\% & 44\% & +146\% & +50\% & 55\% & -22\% & +5\% \\
Display & 30\% & +4\% & -6\% & 37\% & +73\% & +7\% & 29\% & -25\% & +91\% \\
Lamp & 62\% & +13\% & +5\% & 40\% & +94\% & +22\% & 49\% & -23\% & +15\% \\
Speakers & 47\% & +3\% & -1\% & 34\% & +24\% & -1\% & 32\% & -15\% & +44\% \\
Rifle & 19\% & +19\% & -14\% & 21\% & +91\% & -13\% & 32\% & -35\% & +158\% \\
Sofa & 35\% & +5\% & -2\% & 26\% & +14\% & -10\% & 26\% & -17\% & +115\% \\
Table & 48\% & +9\% & +2\% & 45\% & +211\% & +76\% & 42\% & -19\% & +26\% \\
Telephone & 20\% & +4\% & -11\% & 17\% & +21\% & -28\% & 18\% & -22\% & +585\% \\
Watercraft & 47\% & +8\% & -2\% & 35\% & +66\% & +5\% & 31\% & -20\% & +92\

In [77]:
# appendix raw tables

for method in method_paths_dict:
    
    for setting in method_paths_dict[method]["available_settings"]:
        print("{} -- {}".format(method, setting))
        quant_results_df_mean = pd.DataFrame()
        
        for class_id in method_paths_dict[method]["class_ids"]:
            eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
            eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

            combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after)
            quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
            quant_result_dict_mean["instance"] = method_paths_dict[method]["name_cvt_dict"][class_id]

            for metric in ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]:
                if metric in ["CD"]:
                    num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
                    avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean()
                    num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
                    avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
                else:
                    num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
                    avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
                    num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
                    avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean()

                quant_result_dict_mean["{}_P_up".format(metric)] =  num_improved/(num_improved+num_worse)
                quant_result_dict_mean["{}_C_up".format(metric)] =  avg_improve_amt
                quant_result_dict_mean["{}_P_d".format(metric)] =  num_worse/(num_improved+num_worse)
                quant_result_dict_mean["{}_C_d".format(metric)] =  avg_worse_amt


            quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

        quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Before".format(metric): "{}_B".format(metric) for metric in metrics})
        quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_After".format(metric): "{}_A".format(metric) for metric in metrics})
        quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Delta".format(metric): "{}_C".format(metric) for metric in metrics})
        readable_quant_results_df_mean = make_df_readable(quant_results_df_mean)
        df_order = ["instance"]+["{}_{}".format(metric, i) for metric in metrics for i in ["B", "A", "C", "P_up", "C_up", "P_d", "C_d"]]
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(readable_quant_results_df_mean[df_order])
            display(readable_quant_results_df_mean[df_order].transpose())
            
        print_latex(readable_quant_results_df_mean[df_order].transpose())

    break

atlasnet -- gt_pose


,instance,2D_IoU_B,2D_IoU_A,2D_IoU_C,2D_IoU_P_up,2D_IoU_C_up,2D_IoU_P_d,2D_IoU_C_d,MV_IoU_B,MV_IoU_A,MV_IoU_C,MV_IoU_P_up,MV_IoU_C_up,MV_IoU_P_d,MV_IoU_C_d,3D_IoU_B,3D_IoU_A,3D_IoU_C,3D_IoU_P_up,3D_IoU_C_up,3D_IoU_P_d,3D_IoU_C_d,CD_B,CD_A,CD_C,CD_P_up,CD_C_up,CD_P_d,CD_C_d
0,Airplane,0.38,0.54,+44%,96%,+47%,4%,-13%,0.36,0.43,+21%,94%,+23%,6%,-4%,0.06,0.11,+121%,91%,+136%,9%,-20%,0.10,0.05,-59%,95%,-64%,5%,+44%
1,Bench,0.58,0.71,+27%,91%,+31%,9%,-11%,0.50,0.53,+7%,84%,+9%,16%,-3%,0.05,0.07,+44%,70%,+72%,30%,-21%,0.12,0.09,-23%,84%,-33%,16%,+25%
2,Cabinet,0.71,0.81,+15%,94%,+17%,6%,-3%,0.69,0.73,+6%,92%,+6%,8%,-1%,0.14,0.16,+18%,72%,+29%,28%,-11%,0.07,0.05,-33%,93%,-37%,7%,+20%
3,Car,0.52,0.74,+48%,99%,+48%,1%,-3%,0.50,0.62,+25%,100%,+25%,0%,-1%,0.07,0.11,+77%,95%,+82%,5%,-13%,0.24,0.06,-75%,100%,-75%,0%,+4%
4,Chair,0.54,0.62,+14%,86%,+18%,14%,-7%,0.50,0.54,+7%,89%,+8%,11%,-2%,0.08,0.12,+55%,84%,+68%,16%,-16%,0.12,0.06,-46%,97%,-48%,3%,+20%
5,Display,0.70,0.79,+15%,95%,+16%,5%,-7%,0.63,0.66,+5%,87%,+6%,13%,-2%,0.10,0.13,+43%,75%,+62%,25%,-16%,0.11,0.07,-26%,83%,-36%,17%,+21%
6,Lamp,0.47,0.62,+39%,92%,+43%,8%,-15%,0.45,0.51,+16%,93%,+18%,7%,-10%,0.07,0.10,+97%,78%,+129%,22%,-20%,0.27,0.14,-47%,93%,-53%,7%,+35%
7,Speakers,0.71,0.80,+15%,91%,+17%,9%,-4%,0.70,0.74,+6%,92%,+7%,8%,-2%,0.14,0.16,+21%,77%,+31%,23%,-15%,0.11,0.07,-35%,94%,-40%,6%,+27%
8,Rifle,0.37,0.54,+56%,96%,+59%,4%,-15%,0.33,0.43,+30%,99%,+31%,1%,-4%,0.08,0.13,+121%,94%,+130%,6%,-15%,0.15,0.07,-64%,98%,-66%,2%,+39%
9,Sofa,0.69,0.80,+19%,91%,+21%,9%,-4%,0.65,0.68,+5%,89%,+6%,11%,-3%,0.16,0.17,+20%,67%,+38%,33%,-15%,0.12,0.07,-36%,96%,-39%,4%,+18%


,0,1,2,3,4,5,6,7,8,9,10,11,12,mean
instance,Airplane,Bench,Cabinet,Car,Chair,Display,Lamp,Speakers,Rifle,Sofa,Table,Telephone,Watercraft,NaN
2D_IoU_B,0.38,0.58,0.71,0.52,0.54,0.7,0.47,0.71,0.37,0.69,0.6,0.7,0.47,0.57
2D_IoU_A,0.54,0.71,0.81,0.74,0.62,0.79,0.62,0.8,0.54,0.8,0.75,0.81,0.66,0.71
2D_IoU_C,+44%,+27%,+15%,+48%,+14%,+15%,+39%,+15%,+56%,+19%,+28%,+18%,+54%,+30%
2D_IoU_P_up,96%,91%,94%,99%,86%,95%,92%,91%,96%,91%,95%,98%,95%,94%
2D_IoU_C_up,+47%,+31%,+17%,+48%,+18%,+16%,+43%,+17%,+59%,+21%,+29%,+19%,+57%,+32%
2D_IoU_P_d,4%,9%,6%,1%,14%,5%,8%,9%,4%,9%,5%,2%,5%,6%
2D_IoU_C_d,-13%,-11%,-3%,-3%,-7%,-7%,-15%,-4%,-15%,-4%,-6%,-7%,-6%,-8%
MV_IoU_B,0.36,0.5,0.69,0.5,0.5,0.63,0.45,0.7,0.33,0.65,0.53,0.63,0.46,0.53
MV_IoU_A,0.43,0.53,0.73,0.62,0.54,0.66,0.51,0.74,0.43,0.68,0.57,0.69,0.55,0.59


Airplane & Bench & Cabinet & Car & Chair & Display & Lamp & Speakers & Rifle & Sofa & Table & Telephone & Watercraft & Mean \\
0.38 & 0.58 & 0.71 & 0.52 & 0.54 & 0.7 & 0.47 & 0.71 & 0.37 & 0.69 & 0.6 & 0.7 & 0.47 & 0.57 \\
0.54 & 0.71 & 0.81 & 0.74 & 0.62 & 0.79 & 0.62 & 0.8 & 0.54 & 0.8 & 0.75 & 0.81 & 0.66 & 0.71 \\
+44\% & +27\% & +15\% & +48\% & +14\% & +15\% & +39\% & +15\% & +56\% & +19\% & +28\% & +18\% & +54\% & +30\% \\
96\% & 91\% & 94\% & 99\% & 86\% & 95\% & 92\% & 91\% & 96\% & 91\% & 95\% & 98\% & 95\% & 94\% \\
+47\% & +31\% & +17\% & +48\% & +18\% & +16\% & +43\% & +17\% & +59\% & +21\% & +29\% & +19\% & +57\% & +32\% \\
4\% & 9\% & 6\% & 1\% & 14\% & 5\% & 8\% & 9\% & 4\% & 9\% & 5\% & 2\% & 5\% & 6\% \\
-13\% & -11\% & -3\% & -3\% & -7\% & -7\% & -15\% & -4\% & -15\% & -4\% & -6\% & -7\% & -6\% & -8\% \\
0.36 & 0.5 & 0.69 & 0.5 & 0.5 & 0.63 & 0.45 & 0.7 & 0.33 & 0.65 & 0.53 & 0.63 & 0.46 & 0.53 \\
0.43 & 0.53 & 0.73 & 0.62 & 0.54 & 0.66 & 0.51 & 0.74 & 0.43 & 0.68 & 0

In [112]:
i = 0
first = True
print("\\hline")
print(" & & Airplane & Bench & Cabinet & Car & Chair & Display & Lamp & Speakers & Rifle & Sofa & Table & Telephone & Watercraft & Mean \\\\")
for index, row in readable_quant_results_df_mean[df_order].transpose().iterrows():
    if first:
        first = False
        continue
    row_list = [str(x) for x in row.to_list()]
    if i%7==0: row_list.insert(0,"Before")
    elif i%7==1: row_list.insert(0,"After")
    elif i%7==2: row_list.insert(0,"P.C.")
    elif i%7==3: row_list.insert(0,"Prop. $\\uparrow$")
    elif i%7==4: row_list.insert(0,"Amt.  $\\uparrow$")
    elif i%7==5: row_list.insert(0,"Prop.  $\\downarrow$")
    elif i%7==6: row_list.insert(0,"Amt. $\\downarrow$")
    
    if i==0:
        row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{2D IoU}}}")
    elif i==7:
        row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{MV IoU}}}")
    elif i==14:
        row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{3D IoU}}}")
    elif i==21:
        row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{CD}}}")
    else:
        row_list.insert(0," ")
    
    latex_exp = " & ".join(row_list)
    latex_exp += " \\\\"
    
    latex_exp = latex_exp.replace("%", "\\%").replace("nan", "Mean")
    if i % 7 == 0:
        print("\cline{1-16}")
    elif(i%7==3 or i%7==5):
        print("\cline{2-16}")
    print(latex_exp)
    i+=1
print("\\hline")

\hline
 & & Airplane & Bench & Cabinet & Car & Chair & Display & Lamp & Speakers & Rifle & Sofa & Table & Telephone & Watercraft & Mean \\
\cline{1-16}
\parbox[t]{3mm}{\multirow{7}{*}{\rotatebox[origin=c]{90}{2D IoU}}} & Before & 0.38 & 0.58 & 0.71 & 0.52 & 0.54 & 0.7 & 0.47 & 0.71 & 0.37 & 0.69 & 0.6 & 0.7 & 0.47 & 0.57 \\
  & After & 0.54 & 0.71 & 0.81 & 0.74 & 0.62 & 0.79 & 0.62 & 0.8 & 0.54 & 0.8 & 0.75 & 0.81 & 0.66 & 0.71 \\
  & P.C. & +44\% & +27\% & +15\% & +48\% & +14\% & +15\% & +39\% & +15\% & +56\% & +19\% & +28\% & +18\% & +54\% & +30\% \\
\cline{2-16}
  & Prop. $\uparrow$ & 96\% & 91\% & 94\% & 99\% & 86\% & 95\% & 92\% & 91\% & 96\% & 91\% & 95\% & 98\% & 95\% & 94\% \\
  & Amt.  $\uparrow$ & +47\% & +31\% & +17\% & +48\% & +18\% & +16\% & +43\% & +17\% & +59\% & +21\% & +29\% & +19\% & +57\% & +32\% \\
\cline{2-16}
  & Prop.  $\downarrow$ & 4\% & 9\% & 6\% & 1\% & 14\% & 5\% & 8\% & 9\% & 4\% & 9\% & 5\% & 2\% & 5\% & 6\% \\
  & Amt. $\downarrow$ & -13\% & -11\% & -3\% 

In [ ]:

    #print("\n")
    #print("most improved: {}".format(delta_df.sort_values("CD", ascending=True)["instance"].to_list()[:20]))
    #with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #    display(combined_df.sort_values("3D_IoU_Delta", ascending=False))
    #    display(delta_df.sort_values("3D_IoU", ascending=False))
    #    display(results_df_before.sort_values("3D_IoU", ascending=True))  
    